<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [CeRAS reference case](http://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad import api
import logging
from fastoad.utils.postprocessing import VariableViewer
import shutil

DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.toml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'CeRAS01_baseline.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


After defining a configuration file name, we can ask FAST-OAD to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

# The above generated configuration uses the quick and simple Breguet module to compute performances. 
# If you want to use the more accurate, much more CPU-costly time-step integration, uncomment and run the following line:
#shutil.copy(pth.join(DATA_FOLDER_PATH, "oad_process_timestep_mission.toml"), CONFIGURATION_FILE)

INFO    : Sample configuration written in workdir\oad_process.toml


You can now checkout the generated [configuration file](./workdir/oad_process.toml).

In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in D:\s.delbecq\Documents\6-Repositories\FAST-OAD\src\fastoad\notebooks\tutorial\workdir\problem_inputs.xml


'D:\\s.delbecq\\Documents\\6-Repositories\\FAST-OAD\\src\\fastoad\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [4]:
api.list_variables(CONFIGURATION_FILE)

,I/O,Name,Description
0,IN,data:TLAR:NPAX,"top-level requirement: number of passengers, assuming a classic eco/business class repartition"
1,IN,data:TLAR:approach_speed,top-level requirement: approach speed
2,IN,data:TLAR:cruise_mach,top-level requirement: cruise Mach number
3,IN,data:TLAR:range,top-level requirement: design range
4,IN,data:aerodynamics:aircraft:landing:CL_max_clean_2D,
5,IN,data:aerodynamics:aircraft:takeoff:mach,considered Mach number for takeoff phase
6,IN,data:geometry:cabin:aisle_width,width of aisles
7,IN,data:geometry:cabin:containers:count_by_row,number of cargo containers along width
8,IN,data:geometry:cabin:crew_count:technical,number of technical crew members
9,IN,data:geometry:cabin:exit_width,width of exits


Another useful feature is to list the modules of the model defined in the configuration file:

In [5]:
api.list_systems(CONFIGURATION_FILE)

== AVAILABLE SYSTEM IDENTIFIERS ====================================================================
----------------------------------------------------------------------------------------------------
  IDENTIFIER:   fastoad.aerodynamics.highspeed.legacy
  PATH:         d:\s.delbecq\documents\6-repositories\fast-oad\src\fastoad\models\aerodynamics\aerodynamics_high_speed.py
  DOMAIN:       Aerodynamics
  DESCRIPTION:  
    Computes aerodynamic polar of the aircraft in cruise conditions.

    Drag contributions of each part of the aircraft are computed though analytical
    models.

----------------------------------------------------------------------------------------------------
  IDENTIFIER:   fastoad.aerodynamics.landing.legacy
  PATH:         d:\s.delbecq\documents\6-repositories\fast-oad\src\fastoad\models\aerodynamics\aerodynamics_landing.py
  DOMAIN:       Aerodynamics
  DESCRIPTION:  
    Computes aerodynamic characteristics at landing.

    - Computes CL and CD increments du

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [6]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in D:\s.delbecq\Documents\6-Repositories\FAST-OAD\src\fastoad\notebooks\tutorial\workdir\n2.html


Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [7]:
# XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
# api.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
# from IPython.display import IFrame
# IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### CeRAS
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

*(This run should take around 10 seconds)*

In [8]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Mission computation - iteration 0 : Using initializer computation. OTHER ITERATIONS NEEDED.
INFO    : Mission computation - iteration 1 : Using mission definition.
INFO    : Mission computation - iteration 2 : Using mission definition.
INFO    : Mission computation - iteration 3 : Using mission definition.
INFO    : Mission computation - iteration 4 : Using mission definition.
INFO    : Mission computation - iteration 5 : Using mission definition.
INFO    : Mission computation - iteration 6 : Using mission definition.
INFO    : Mission computation - iteration 7 : Using mission definition.
INFO    : Mission computation - iteration 8 : Using mission definition.
INFO    : Mission computation - iteration 9 : Using mission definition.
INFO    : Mission computation - iteration 10 : Using mission definition.
INFO    : Mission computation - iteration 11 : Using mission definition.
INFO    : Mission computation - iteration 12 : Using mission definition.
INFO    : Mission computation -

NL: NLBGS Converged in 36 iterations


INFO    : Computation finished after 10.46 seconds
INFO    : Problem outputs written in D:\s.delbecq\Documents\6-Repositories\FAST-OAD\src\fastoad\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [9]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_2750nm_mda.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OUTPUT_FILE)

'workdir\\problem_outputs_CeRAS_2750nm_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [10]:
api.variable_viewer(OUTPUT_FILE)

### CeRAS for 800 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 800 nm range instead of the 2400 nm of the CeRAS. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [11]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

*(This run should take around 10 seconds)*

In [12]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Mission computation - iteration 0 : Using initializer computation. OTHER ITERATIONS NEEDED.
INFO    : Mission computation - iteration 1 : Using mission definition.
INFO    : Mission computation - iteration 2 : Using mission definition.
INFO    : Mission computation - iteration 3 : Using mission definition.
INFO    : Mission computation - iteration 4 : Using mission definition.
INFO    : Mission computation - iteration 5 : Using mission definition.
INFO    : Mission computation - iteration 6 : Using mission definition.
INFO    : Mission computation - iteration 7 : Using mission definition.
INFO    : Mission computation - iteration 8 : Using mission definition.
INFO    : Mission computation - iteration 9 : Using mission definition.
INFO    : Mission computation - iteration 10 : Using mission definition.
INFO    : Mission computation - iteration 11 : Using mission definition.
INFO    : Mission computation - iteration 12 : Using mission definition.
INFO    : Mission computation -

NL: NLBGS Converged in 31 iterations


INFO    : Computation finished after 8.45 seconds
INFO    : Problem outputs written in D:\s.delbecq\Documents\6-Repositories\FAST-OAD\src\fastoad\notebooks\tutorial\workdir\problem_outputs.xml


Let's save again these new results, for post-processing them in next notebook.

In [13]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_800nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, CeRAS_800nm_OUTPUT_FILE)

'workdir\\problem_outputs_CeRAS_800nm_mda.xml'

## 3. Running your first MDO

## CeRAS 2750 nm optimization

It is possible to run an optimization problem on top of your analysis. Thus, you can choose to set some global inputs as design variables, and global outputs as constraints or the objective. Here is an example given in the default configuration file that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the wing aspect ratio (design variables),
- subject to a wing span limit (constraints).

The last part of the configuration file .toml is where this optimization problem is defined:

```toml
[optimization]  # This section is needed only if optimization process is run
    [[optimization.design_var]]
        name = "data:geometry:wing:aspect_ratio"
        lower = 9.0
        upper = 18.0

    [[optimization.constraint]]
        name = "data:geometry:wing:span"
        upper = 60.0

    [[optimization.objective]]
        name = "data:mission:sizing:fuel"
        scaler = 1e-4
```

Note that we need to use [scaling](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/scaling.html) on the objective (fuel mass for the mission) so that it has the same order of magnitude than the design variable (wing aspect ratio) by adding the line `scaler = 1e-4`. Without this the optimizer might not detect that the steps it makes on the wing aspect ratio has an effect on decreasing the fuel mass and could result in an early stop.


*(This run should take around 3 minutes)*

In [14]:
# Set back the inputs from the reference CeRAS 2750 nm
api.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in D:\s.delbecq\Documents\6-Repositories\FAST-OAD\src\fastoad\notebooks\tutorial\workdir\problem_inputs.xml


'D:\\s.delbecq\\Documents\\6-Repositories\\FAST-OAD\\src\\fastoad\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [15]:
api.optimization_viewer(CONFIGURATION_FILE)

In [16]:
optim_problem = api.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Mission computation - iteration 0 : Using initializer computation. OTHER ITERATIONS NEEDED.
INFO    : Mission computation - iteration 1 : Using mission definition.
INFO    : Mission computation - iteration 2 : Using mission definition.
INFO    : Mission computation - iteration 3 : Using mission definition.
INFO    : Mission computation - iteration 4 : Using mission definition.
INFO    : Mission computation - iteration 5 : Using mission definition.
INFO    : Mission computation - iteration 6 : Using mission definition.
INFO    : Mission computation - iteration 7 : Using mission definition.
INFO    : Mission computation - iteration 8 : Using mission definition.
INFO    : Mission computation - iteration 9 : Using mission definition.
INFO    : Mission computation - iteration 10 : Using mission definition.
INFO    : Mission computation - iteration 11 : Using mission definition.
INFO    : Mission computation - iteration 12 : Using mission definition.
INFO    : Mission computation -

NL: NLBGS Converged in 36 iterations


INFO    : Mission computation - iteration 36 : Using mission definition.
INFO    : Mission computation - iteration 37 : Using mission definition.


NL: NLBGS Converged in 2 iterations


INFO    : Mission computation - iteration 38 : Using mission definition.
INFO    : Mission computation - iteration 39 : Using mission definition.
INFO    : Mission computation - iteration 40 : Using mission definition.
INFO    : Mission computation - iteration 41 : Using mission definition.
INFO    : Mission computation - iteration 42 : Using mission definition.
INFO    : Mission computation - iteration 43 : Using mission definition.
INFO    : Mission computation - iteration 44 : Using mission definition.
INFO    : Mission computation - iteration 45 : Using mission definition.
INFO    : Mission computation - iteration 46 : Using mission definition.
INFO    : Mission computation - iteration 47 : Using mission definition.
INFO    : Mission computation - iteration 48 : Using mission definition.
INFO    : Mission computation - iteration 49 : Using mission definition.
INFO    : Mission computation - iteration 50 : Using mission definition.
INFO    : Mission computation - iteration 51 : Usin

...

INFO    : Mission computation - iteration 650 : Using mission definition.
INFO    : Mission computation - iteration 651 : Using mission definition.
INFO    : Mission computation - iteration 652 : Using mission definition.


NL: NLBGS Converged in 3 iterations


INFO    : Mission computation - iteration 653 : Using mission definition.
INFO    : Mission computation - iteration 654 : Using mission definition.
INFO    : Mission computation - iteration 655 : Using mission definition.


NL: NLBGS Converged in 3 iterations
Optimization Complete
-----------------------------------


INFO    : Computation finished after 170.02 seconds
INFO    : Problem outputs written in D:\s.delbecq\Documents\6-Repositories\FAST-OAD\src\fastoad\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results:

In [17]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_CeRAS_2750nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

'workdir\\problem_outputs_CeRAS_2750nm_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [18]:
api.optimization_viewer(CONFIGURATION_FILE)

We see that the optimizer finds a compromise between a smaller induced drag (higher aspect ratio) and a heavier wing.

**Note: Finding such compromise on real aircraft is still a subject of research. Please take this result as informational.**

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [19]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')

In [20]:
api.variable_viewer(RESULT_FILE)